# Métrica 2_1_04 — Pareja y/o dependientes que son personas servidoras públicas con inversiones o cuentas

In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from pymongo.errors import ConnectionFailure, OperationFailure, BulkWriteError
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "2_1_04_PAREJA_DEPENDIENTES_SP_CON_TITULARIDAD"

# --- FUNCIONES AUXILIARES ---

def pareja_servidor_publico(doc):
    try:
        pareja = (
            doc.get("declaracion", {})
               .get("situacionPatrimonial", {})
               .get("datosPareja", {})
        )
        actividad = pareja.get("actividadLaboral", "")
        if isinstance(actividad, str) and "SERVIDOR" in actividad.upper():
            return True
        return False
    except Exception:
        return False

def dependientes_servidores_publicos(doc):
    try:
        dependientes = (
            doc.get("declaracion", {})
               .get("situacionPatrimonial", {})
               .get("datosDependientesEconomicos", {})
               .get("dependienteEconomico", [])
        )
        if not isinstance(dependientes, list):
            return False
        for d in dependientes:
            act = d.get("actividadLaboral", "")
            if isinstance(act, str) and "SERVIDOR" in act.upper():
                return True
        return False
    except Exception:
        return False

def titular_inversion_pareja_o_dependiente(doc):
    try:
        inversiones = (
            doc.get("declaracion", {})
               .get("situacionPatrimonial", {})
               .get("inversiones", {})
               .get("inversion", [])
        )
        if not isinstance(inversiones, list):
            return False
        for inv in inversiones:
            titular = inv.get("titular", {}).get("valor", "").upper()
            if titular in ["PAREJA", "DEPENDIENTE", "PAREJA O DEPENDIENTE"]:
                return True
        return False
    except Exception:
        return False


# --- FUNCIÓN PRINCIPAL ---

def evaluar_metrica(doc):
    pareja_sp = pareja_servidor_publico(doc)
    dep_sp = dependientes_servidores_publicos(doc)
    titular_rel = titular_inversion_pareja_o_dependiente(doc)

    # Caso 1: No hay pareja/dependiente servidor público
    if not pareja_sp and not dep_sp:
        return "N/A"

    # Caso 2: Hay pareja/dependiente SP y figura como titular
    if (pareja_sp or dep_sp) and titular_rel:
        return "CUMPLE"

    # Caso 3: Hay pareja/dependiente SP pero no figura como titular
    if (pareja_sp or dep_sp) and not titular_rel:
        return "NO_CUMPLE"

    # Caso 4: Faltan datos obligatorios o públicos
    return "SIN_DATO"


# --- PROCESAMIENTO EN LOTE ---

def procesar_metrica_2_1_04():
    client = None
    operaciones = []
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "N/A": 0}
    total = 0

    try:
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        source = db[SOURCE_COLLECTION_NAME]
        target = db[METRICS_COLLECTION_NAME]

        print(f"Procesando métrica {METRIC_ID}...")

        cursor = source.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosPareja.actividadLaboral": 1,
            "declaracion.situacionPatrimonial.datosDependientesEconomicos.dependienteEconomico.actividadLaboral": 1,
            "declaracion.situacionPatrimonial.inversiones.inversion.titular": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1
            resultado = evaluar_metrica(doc)
            resultados[resultado] += 1

            operaciones.append(UpdateOne(
                {"_id": doc["_id"]},
                {"$set": {METRIC_ID: resultado}},
                upsert=True
            ))

            if len(operaciones) == 1000:
                target.bulk_write(operaciones)
                operaciones.clear()
                print(f"  > Procesados {total} documentos...")

        if operaciones:
            target.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Documentos procesados: {total}")
        for k, v in resultados.items():
            print(f"  > {k}: {v}")

    except Exception as e:
        print(f"Error general: {e}")
        traceback.print_exc()
    finally:
        if client:
            client.close()
            print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_2_1_04()


Procesando métrica 2_1_04_PAREJA_DEPENDIENTES_SP_CON_TITULARIDAD...


c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


  > Procesados 1000 documentos...
  > Procesados 2000 documentos...
  > Procesados 3000 documentos...
  > Procesados 4000 documentos...
  > Procesados 5000 documentos...
  > Procesados 6000 documentos...
  > Procesados 7000 documentos...
  > Procesados 8000 documentos...
  > Procesados 9000 documentos...
  > Procesados 10000 documentos...
  > Procesados 11000 documentos...
  > Procesados 12000 documentos...
  > Procesados 13000 documentos...
  > Procesados 14000 documentos...
  > Procesados 15000 documentos...
  > Procesados 16000 documentos...
  > Procesados 17000 documentos...
  > Procesados 18000 documentos...
  > Procesados 19000 documentos...
  > Procesados 20000 documentos...
  > Procesados 21000 documentos...
  > Procesados 22000 documentos...
  > Procesados 23000 documentos...
  > Procesados 24000 documentos...
  > Procesados 25000 documentos...
  > Procesados 26000 documentos...
  > Procesados 27000 documentos...
  > Procesados 28000 documentos...
  > Procesados 29000 document

KeyboardInterrupt: 